In [3]:
import os
from dotenv import load_dotenv
from typing import Literal,Union
import pandas as pd
import time

In [7]:
df = pd.read_csv('structured_jobs_data.csv')

In [8]:
df

,reasoning,company,job_title,location,domain,domain_specific_skills,work_model,min_experience,max_experience,number_of_employeees,...,max_salary,no_of_applicants,company_type,key_responsibilities,technical_requirements,education,benefits_culture,unique_aspects,application_link,applicant_insights
0,The job description provides details for a Sof...,Tower Research Capital,Junior Software Engineer,"Gurgaon, Haryana, India",Financial Technology,"SQL, NoSQL",Hybrid,0,3,"1,001-5,000",...,NaN,100,"Financial Services, 1,001-5,000 employees",Developing the firm’s Post Trade Platform tool...,"Knowledge of Python and Java, shell scripts an...",Bachelors,"Tower fosters a culture where smart, driven pe...",Focus on a collaborative and welcoming culture...,NaN,72% of applicants are entry level and 69% have...
1,I have extracted the information from the prov...,D. E. Shaw India Private Limited,Senior Software Engineer,"Gurugram, Haryana, India",AI,"Python, data analysis",Hybrid,0,10,"1,001-5,000",...,NaN,100,Large financial services firm,"Executing projects, collaborating with busines...","C++, Java, Python, React, Redux, RESTful Web S...",Bachelors,"Excellent benefits, casual, collegial working ...","Entrepreneurial culture, building internal too...",1.0,72% of applicants have a Bachelor of Technolog...
2,The job description is for an Engineer I posit...,American Express,Engineer I,"Gurugram, Haryana, India",AI/ML,"Airflow, Jupyterlab, Jupyterhub, MLFlow, Spark...",Hybrid,2,5,"10,000+",...,NaN,100,Large financial services company,Perform technical aspects of software developm...,Experience in large scale platform engineering...,Bachelors,"Competitive salaries, bonus incentives, financ...","Focus on AI/ML platform engineering, opportuni...",NaN,"83% of applicants have a Bachelor's Degree, 82..."
3,The job description provides a comprehensive o...,Bain & Company,Senior Software Engineer,"Gurugram, Haryana, India","AI, Web Development","C#, T-SQL, JavaScript, HTML, CSS, Bootstrap, R...",Hybrid,3,5,"10,001+",...,NaN,100,"Global consultancy with 10,001+ employees",Developing and updating enterprise application...,".NET & .NET Core, C#, T-SQL, Angular/React, Ja...",Bachelors,Bain & Company is committed to investing in pr...,"The role emphasizes T-shaped skills, contribut...",NaN,68% of applicants have a Bachelor of Technolog...
4,The job posting is for a Software Engineer - P...,Zenarate,Junior-Mid Level Python Engineer,"Gurugram, Haryana, India",AI,"RESTful & GraphQL API design, SQL, NoSQL datab...",Hybrid,2,4,51-200,...,NaN,100,Software Development company with 51-200 emplo...,"Develop microservices using Python, applicatio...","Expert in Python, at least 2+ years knowledge ...",Bachelors,"Competitive Salary, Comprehensive Benefits (he...",Zenarate is a leading AI Simulation + Coaching...,NaN,57% of applicants have a Bachelor of Technolog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,The job description provides detailed informat...,BlackRock,Senior Data Scientist Engineer,"Gurgaon, Haryana, India",AI,"Python, Scala, Spark, Hadoop, GCP, Azure, Mach...",Hybrid,4,10,"10,000+",...,NaN,100,Global asset manager and technology provider,"Design, develop, and maintain data pipelines t...","Strong background in Mathematics, Statistics, ...",Bachelors,"Strong retirement plan, tuition reimbursement,...",Opportunity to gain in-depth knowledge of the ...,NaN,21% employee growth. 38% of applicants have a ...
351,The job description provides details for a Qua...,BlackRock,junior quality assurance engineer,"Gurgaon, Haryana, India",AI web development,"Java, Selenium, Cypress, Cucumber, Jenkins, Az...",Hybrid,1,4,"10,000+",...,NaN,100,Global asset manager and technology provider,Become a Subject Matter Expert on applications...,"Java, Selenium, Cypress, Cucumber, Jenkins, Az...",Bachelors,"Strong retirement plan, tuition reimbursement,...",Opportunity to work on BlackRock's investment ...,NaN,76% of applicants are

In [8]:
load_dotenv()
# lm = dspy.LM("groq/llama-3.3-70b-versatile", api_key= os.getenv("GROQ_API_KEY"))
lm = dspy.LM('ollama_chat/gemma3:27b-it-qat', api_base='http://localhost:11435', api_key='',model_type="chat")
dspy.configure(lm=lm)

In [9]:
test = lm("this is a test!", temperature=0.7) 

21:18:01 - LiteLLM:ERROR: caching.py:633 - LiteLLM Cache: Excepton add_cache: __annotations__
Traceback (most recent call last):
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 628, in add_cache
    cache_key, cached_data, kwargs = self._add_cache_logic(
                                     ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 612, in _add_cache_logic
    raise e
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 592, in _add_cache_logic
    cache_key = self.get_cache_key(**kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 254, in get_cache_key
    combined_kwargs = self._get_relevant_args_to_use_for_cache_

In [10]:
test

["Okay, I see this is a test! \n\nIs there anything specific you want me to *do* with this test? For example, are you testing:\n\n* **My ability to recognize the start/end tags?**\n* **My responsiveness?**\n* **Something else?**\n\nJust let me know what you'd like me to demonstrate.\n"]

In [ ]:


class JobDetails(dspy.Signature):
    """Extract detailed job information from a listing."""
    job_text: str = dspy.InputField()
    company: str = dspy.OutputField(desc="Name of the company")
    job_title: str = dspy.OutputField(desc="standardized job format in the sequence seniority(eg junior,senior) , title(eg full-stack,web,ml), role (engineer,intern) ")
    location: str = dspy.OutputField(desc="Job location")
    domain: str = dspy.OutputField(desc="Domain of work AI web,development,full-stack")
    domain_specific_skills : str = dspy.OutputField(desc="Technical skills only relevant to the current domain eg opencv,pandas,numpy,tensorflow for AI")
    work_model: str = dspy.OutputField(desc="Work model (e.g., Hybrid, Full-time)")
    min_experience: int = dspy.OutputField(desc="minimum experience level in years")
    max_experience: int = dspy.OutputField(desc="maximum experience level in years")
    number_of_employeees : str = dspy.OutputField(desc='number of employees working in company')
    weeks_since_posting: int = dspy.OutputField(desc="weeks passed since posting date")
    min_salary: Union[None, int] = dspy.OutputField(desc="minimum pay range")
    max_salary: Union[None, int] = dspy.OutputField(desc="maximum pay range")
    no_of_applicants : int = dspy.OutputField(desc="number of people that have clicked apply")
    company_type: str = dspy.OutputField(desc="Type of company, size, and funding stage")
    key_responsibilities: str = dspy.OutputField(desc="Key responsibilities and tasks for the role")
    technical_requirements: str = dspy.OutputField(desc="Required technical skills and technologies")
    education: Literal['Bachelors','Masters','Phd'] = dspy.OutputField(desc="minimum Required educational background")
    benefits_culture: str = dspy.OutputField(desc="Company culture, benefits, and perks")
    unique_aspects: str = dspy.OutputField(desc="Unique aspects of the role or company")
    application_link: Union[None, int] = dspy.OutputField(desc="How to apply for the job")
    applicant_insights: str = dspy.OutputField(desc="Insights on typical applicants and experience levels")


In [ ]:
job_parser = dspy.ChainOfThought(JobDetails)
text = '''Tower Research Capital
Share
Show more options
Software Engineer I 
Gurgaon, Haryana, India · Reposted 2 weeks ago · Over 100 people clicked apply
Promoted by hirer · Responses managed off LinkedIn
Hybrid
Apply
Save
Save Software Engineer I  at Tower Research Capital
Update your profile to assess your job match
Update profile
Get personalized tips to stand out to hirers
Find jobs where you’re a top applicant and tailor your resume with the help of AI.
About the job
Tower Research Capital is a leading quantitative trading firm founded in 1998. Tower has built its business on a high-performance platform and independent trading teams. We have a 25+ year track record of innovation and a reputation for discovering unique market opportunities.

Tower is home to some of the world’s best systematic trading and engineering talent. We empower portfolio managers to build their teams and strategies independently while providing the economies of scale that come from a large, global organization.

Engineers thrive at Tower while developing electronic trading infrastructure at a world class level. Our engineers solve challenging problems in the realms of low-latency programming, FPGA technology, hardware acceleration and machine learning. Our ongoing investment in top engineering talent and technology ensures our platform remains unmatched in terms of functionality, scalability and performance.

At Tower, every employee plays a role in our success. Our Business Support teams are essential to building and maintaining the platform that powers everything we do — combining market access, data, compute, and research infrastructure with risk management, compliance, and a full suite of business services. Our Business Support teams enable our trading and engineering teams to perform at their best.

At Tower, employees will find a stimulating, results-oriented environment where highly intelligent and motivated colleagues inspire each other to reach their greatest potential.

Responsibilities:

Developing the firm’s Post Trade Platform tools - including reference data management, trade capture, reconciliation, P&L, trade cost management, surveillance, compliance and regulatory functions.
Working with appropriate stakeholders to determine user requirements for a set of features.
Working in collaboration with teams across the organization to develop reliable, scalable, and high-performance, software designs.
Responsible for technological innovation in build-out of the new generation of middle office and back-office applications and processes.

Qualifications:

A bachelor’s degree or equivalent in computer science from a top college or university.
0-3 years of relevant experience.
Good problem-solving skills.
Knowledge of Python and Java, shell scripts and linux system
Knowledge of SQL (i.e. MySQL, PostGres) and NoSQL (i.e. MongoDB, ElasticSearch) databases are highly desirable.
Past role(s) within electronic trading and a thorough understanding of financial instruments including equities, commodities, credit, interest rates, foreign exchange, cryptocurrencies, futures, forwards, and options will be a plus
Strong communication skills and fluent in English

Benefits:

Tower’s headquarters are in the historic Equitable Building, right in the heart of NYC’s Financial District and our impact is global, with over a dozen offices around the world.

At Tower, we believe work should be both challenging and enjoyable. That is why we foster a culture where smart, driven people thrive – without the egos. Our open concept workplace, casual dress code, and well-stocked kitchens reflect the value we place on a friendly, collaborative environment where everyone is respected, and great ideas win.

Our benefits include:

Generous paid time off policies
Savings plans and other financial wellness tools available in each region
Hybrid working opportunities
Free breakfast, lunch and snacks daily 
In-office wellness experiences and reimbursement for select wellness expenses (e.g., gym, personal training and more) 
Volunteer opportunities and charitable giving 
Social events, happy hours, treats and celebrations throughout the year
Workshops and continuous learning opportunities

At Tower, you’ll find a collaborative and welcoming culture, a diverse team and a workplace that values both performance and enjoyment. No unnecessary hierarchy. No ego. Just great people doing great work – together.

Tower Research Capital is an equal opportunity employer.
Unlock hiring insights on Tower Research Capital
6%
6
Employee growth
Chart
May 2024
Nov 2024
May 2025
End of interactive chart.
Applicant education level
69%
have a Bachelor of Technology
Applicant seniority level
72%
Entry level applicants
1-month free with 24/7 support. Cancel anytime. We’ll remind you 7 days before your trial ends.
About the company
Tower Research Capital
172,012 followers
Follow
Financial Services 1,001-5,000 employees 1,252 on LinkedIn
Tower Research Capital is a leading quantitative trading firm that has built its business on a high-performance platform and independent trading teams. We have a 25-year track record of innovation and a reputation for discovering unique market opportunities. At Tower, employees find a stimulating, results-oriented environment where highly intelligent and motivated colleagues inspire each other to reach their greatest potential. Tower is headquartered in New York City and employs approximately 1,200 people in more than a dozen offices globally. To learn more, visit www.tower-research.com
…
show more
Interested in working with us in the future?
Members who share that they’re interested in a company may be 2x as likely to get a message from a recruiter than those who don’t. Learn more
Learn more about Interested in working for our company
I’m interested
Company photos
Page 1 of 3
Previous
Next
Show mor

--------------------------------------------------------------------------------'''

NameError: name 'JobDetails' is not defined

In [ ]:
answ = job_parser(job_text = text)

02:21:20 - LiteLLM:ERROR: caching.py:633 - LiteLLM Cache: Excepton add_cache: __annotations__
Traceback (most recent call last):
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 628, in add_cache
    cache_key, cached_data, kwargs = self._add_cache_logic(
                                     ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 612, in _add_cache_logic
    raise e
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 592, in _add_cache_logic
    cache_key = self.get_cache_key(**kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\avira\OneDrive\Desktop\repositories\skill-search\.venv\Lib\site-packages\litellm\caching\caching.py", line 254, in get_cache_key
    combined_kwargs = self._get_relevant_args_to_use_for_cache_

In [25]:
answ.toDict()

{'reasoning': 'The job listing provides detailed information about the role, company, qualifications, responsibilities, benefits, and culture. The key points were extracted to fill in the output fields accordingly.',
 'company': 'Tower Research Capital',
 'job_title': 'Junior Software Engineer',
 'location': 'Gurgaon, Haryana, India',
 'domain': 'financial systems',
 'domain_specific_skills': 'Python, Java, shell scripts, Linux, SQL (MySQL, PostgreSQL), NoSQL (MongoDB, Elasticsearch)',
 'work_model': 'Hybrid',
 'min_experience': 0,
 'max_experience': 3,
 'number_of_employeees': 'Over 1000',
 'weeks_since_posting': 2,
 'min_salary': None,
 'max_salary': None,
 'no_of_applicants': 100,
 'company_type': 'Quantitative Trading Firm (Over 1000 Employees)',
 'key_responsibilities': 'Developing post-trade platform tools, working with stakeholders to determine user requirements, collaborating across teams to develop reliable and scalable software designs, responsible for technological innovatio

In [11]:
import pandas as pd
import time

with open('jobs.txt', 'r', encoding='utf-8') as file:
    jobs = file.read().split('-' * 80)
    first_iter = True

    for idx, job_data in enumerate(jobs):
        try:
            data = job_parser(job_text=job_data)
        except Exception as e:
            print(f'failed to parse index {idx}: {e}, skipping....')
            continue  # Skip this iteration if parsing fails

        time.sleep(60)

        row_df = pd.DataFrame([data.toDict()])  # Create 1-row DataFrame

        if first_iter:
            df = row_df
            first_iter = False
        else:
            df = pd.concat([df, row_df], ignore_index=True)  # assign back to df

df.to_csv('structured_jobs_data.csv', index=False)


failed to parse index 0: name 'job_parser' is not defined, skipping....
failed to parse index 1: name 'job_parser' is not defined, skipping....
failed to parse index 2: name 'job_parser' is not defined, skipping....
failed to parse index 3: name 'job_parser' is not defined, skipping....
failed to parse index 4: name 'job_parser' is not defined, skipping....
failed to parse index 5: name 'job_parser' is not defined, skipping....
failed to parse index 6: name 'job_parser' is not defined, skipping....
failed to parse index 7: name 'job_parser' is not defined, skipping....
failed to parse index 8: name 'job_parser' is not defined, skipping....
failed to parse index 9: name 'job_parser' is not defined, skipping....
failed to parse index 10: name 'job_parser' is not defined, skipping....
failed to parse index 11: name 'job_parser' is not defined, skipping....
failed to parse index 12: name 'job_parser' is not defined, skipping....
failed to parse index 13: name 'job_parser' is not defined, s

NameError: name 'df' is not defined

In [14]:
df = pd.read_csv('structured_jobs_data.csv')
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)d


In [15]:
df

,reasoning,company,job_title,location,domain,domain_specific_skills,work_model,min_experience,max_experience,number_of_employeees,...,max_salary,no_of_applicants,company_type,key_responsibilities,technical_requirements,education,benefits_culture,unique_aspects,application_link,applicant_insights
0,The job description provides details for a Sof...,Tower Research Capital,Junior Software Engineer,"Gurgaon, Haryana, India",Financial Technology,"SQL, NoSQL",Hybrid,0,3,"1,001-5,000",...,NaN,100,"Financial Services, 1,001-5,000 employees",Developing the firm’s Post Trade Platform tool...,"Knowledge of Python and Java, shell scripts an...",Bachelors,"Tower fosters a culture where smart, driven pe...",Focus on a collaborative and welcoming culture...,NaN,72% of applicants are entry level and 69% have...
1,I have extracted the information from the prov...,D. E. Shaw India Private Limited,Senior Software Engineer,"Gurugram, Haryana, India",AI,"Python, data analysis",Hybrid,0,10,"1,001-5,000",...,NaN,100,Large financial services firm,"Executing projects, collaborating with busines...","C++, Java, Python, React, Redux, RESTful Web S...",Bachelors,"Excellent benefits, casual, collegial working ...","Entrepreneurial culture, building internal too...",1.0,72% of applicants have a Bachelor of Technolog...
2,The job description is for an Engineer I posit...,American Express,Engineer I,"Gurugram, Haryana, India",AI/ML,"Airflow, Jupyterlab, Jupyterhub, MLFlow, Spark...",Hybrid,2,5,"10,000+",...,NaN,100,Large financial services company,Perform technical aspects of software developm...,Experience in large scale platform engineering...,Bachelors,"Competitive salaries, bonus incentives, financ...","Focus on AI/ML platform engineering, opportuni...",NaN,"83% of applicants have a Bachelor's Degree, 82..."
3,The job description provides a comprehensive o...,Bain & Company,Senior Software Engineer,"Gurugram, Haryana, India","AI, Web Development","C#, T-SQL, JavaScript, HTML, CSS, Bootstrap, R...",Hybrid,3,5,"10,001+",...,NaN,100,"Global consultancy with 10,001+ employees",Developing and updating enterprise application...,".NET & .NET Core, C#, T-SQL, Angular/React, Ja...",Bachelors,Bain & Company is committed to investing in pr...,"The role emphasizes T-shaped skills, contribut...",NaN,68% of applicants have a Bachelor of Technolog...
4,The job posting is for a Software Engineer - P...,Zenarate,Junior-Mid Level Python Engineer,"Gurugram, Haryana, India",AI,"RESTful & GraphQL API design, SQL, NoSQL datab...",Hybrid,2,4,51-200,...,NaN,100,Software Development company with 51-200 emplo...,"Develop microservices using Python, applicatio...","Expert in Python, at least 2+ years knowledge ...",Bachelors,"Competitive Salary, Comprehensive Benefits (he...",Zenarate is a leading AI Simulation + Coaching...,NaN,57% of applicants have a Bachelor of Technolog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,The job description provides details for a Dat...,NatWest Group,Director Data Engineer,"Gurugram, Haryana, India",AI,"Data modeling, ETL, DevOps, relational databas...",Hybrid,7,10,"10,001+",...,NaN,11,Large banking company with a focus on enterpri...,"Building data solutions, driving DevOps adopti...","Experience with large-scale data, data modelin...",Bachelors,The company emphasizes a dynamic learning cult...,"Focus on enterprise, learning, and climate sol...",NaN,30% of applicants have a Bachelor of Technolog...
313,The job description is for a Senior Engineer-S...,Jacobs,Senior Engineer-Structures,"Gurgaon, Haryana, India",Civil Engineering,"Structural Design, RCC Design, Steel Connectio...",Hybrid,5,10,"10,001+",...,NaN,15,"Business Consulting and Services, 10,001+ empl...","Design of Large Industrial RCC Buildings, Util...","Staad, Etabs, RAM Connections, Idea Statica, S...",Bachelors,"Collaboration, hybrid work policy, employee gr...",Emphasis on collaboration and in-person intera...,NaN,"23% have a Master's D

In [40]:
with open('jobs.txt', 'r', encoding='utf-8') as file:
    jobs = file.read().split('-' * 80)